In [1]:
import pandas as pd
import numpy as np

In [2]:
calendar = pd.read_csv("../sources/calendar/special_days_2020_2025.csv")
transport = pd.read_csv("../sources/transportation/daily_transport_rail.csv")

In [3]:
calendar 

,Subject,Start Date,Category
0,วันขึ้นปีใหม่,2020-01-01,holiday
1,วันเด็ก,2020-01-11,festival
2,วันมาฆบูชา,2020-02-08,holiday
3,วันหยุดชดเชย,2020-02-10,holiday
4,วาเลนไทน์,2020-02-14,festival
...,...,...,...
183,วันพ่อ,2025-12-05,holiday
184,วันรัฐธรรมนูญ,2025-12-10,holiday
185,คริสต์มาสอีฟ,2025-12-24,festival
186,คริสต์มาส,2025-12-25,festival


In [4]:
transport 

,ยานพาหนะ/ท่า,วันที่,ปริมาณ
0,รถไฟฟ้าสายสีน้ำเงิน,2020-01-01,236053.0
1,รถไฟฟ้าสายสีม่วง,2020-01-01,23135.0
2,รถไฟฟ้า ARL,2020-01-01,48790.0
3,รถไฟฟ้า BTS,2020-01-01,461400.0
4,รถไฟฟ้าสายสีน้ำเงิน,2020-01-02,341059.0
...,...,...,...
11166,รถไฟฟ้าสายสีเหลือง,2025-09-03,52235.0
11167,รถไฟฟ้าสายสีชมพู,2025-09-03,75169.0
11168,รถไฟฟ้า ARL,2025-09-03,75562.0
11169,รถไฟฟ้า BTS,2025-09-03,837496.0


In [5]:
# แปลงคอลัมน์วันที่เป็น datetime
transport['วันที่'] = pd.to_datetime(transport['วันที่'])

# หาชื่อคอลัมน์วันที่และประเภทใน calendar
date_col_name = None
category_col_name = None
desc_col_name = None

for col in calendar.columns:
    if 'date' in col.lower() or 'วันที่' in col.lower():
        date_col_name = col
    elif 'category' in col.lower() or 'ประเภท' in col.lower():
        category_col_name = col
    elif 'description' in col.lower() or 'รายละเอียด' in col.lower() or 'ชื่อ' in col.lower():
        desc_col_name = col

calendar[date_col_name] = pd.to_datetime(calendar[date_col_name])

print(f"ใช้คอลัมน์: วันที่={date_col_name}, ประเภท={category_col_name}, รายละเอียด={desc_col_name}")

# ตรวจสอบประเภทที่มีใน Category
print("\nประเภทใน Category:")
print(calendar[category_col_name].value_counts())

# สร้าง is_festival จากคอลัมน์ Category โดยตรง
calendar['is_festival'] = calendar[category_col_name].str.lower().str.contains('festival', na=False)

# ฟังก์ชันแมปประเภทวัน (แก้ไขลำดับการตรวจสอบ)
def map_day_type(row):
    date = row['วันที่']
    
    # เช็ควันพิเศษในปฏิทินก่อน (เพื่อให้ festival มีความสำคัญมากกว่า)
    matched_holiday = calendar[calendar[date_col_name].dt.date == date.date()]
    
    if not matched_holiday.empty:
        if matched_holiday['is_festival'].iloc[0]:
            return 2  # festival
        else:
            return 0  # holiday
    
    # ถ้าไม่พบในปฏิทิน ถึงจะเช็ควันเสาร์-อาทิตย์
    if date.weekday() >= 5:
        return 0  # holiday
    
    return 1  # normal

# แมปประเภทวัน
transport['day_type'] = transport.apply(map_day_type, axis=1)

# แสดงผลการแมป
day_type_counts = transport['day_type'].value_counts().sort_index()
print("จำนวนวันในแต่ละประเภท:")
for day_type, count in day_type_counts.items():
    day_name = {0: 'Holiday', 1: 'Normal', 2: 'Festival'}[day_type]
    print(f"{day_name} ({day_type}): {count:,d} วัน")

print(f"\nจำนวนเทศกาลที่พบใน calendar: {calendar['is_festival'].sum()}")
display(transport.head())

ใช้คอลัมน์: วันที่=Start Date, ประเภท=Category, รายละเอียด=None

ประเภทใน Category:
Category
holiday     164
festival     24
Name: count, dtype: int64
จำนวนวันในแต่ละประเภท:
Holiday (0): 3,808 วัน
Normal (1): 7,247 วัน
Festival (2): 116 วัน

จำนวนเทศกาลที่พบใน calendar: 24


,ยานพาหนะ/ท่า,วันที่,ปริมาณ,day_type
0,รถไฟฟ้าสายสีน้ำเงิน,2020-01-01,236053.0,0
1,รถไฟฟ้าสายสีม่วง,2020-01-01,23135.0,0
2,รถไฟฟ้า ARL,2020-01-01,48790.0,0
3,รถไฟฟ้า BTS,2020-01-01,461400.0,0
4,รถไฟฟ้าสายสีน้ำเงิน,2020-01-02,341059.0,1


In [6]:
# สร้าง df_combined โดยรวมข้อมูลกับ calendar
df_combined = transport.copy()

# สร้างชื่อวันในสัปดาห์ภาษาไทย
day_names_thai = {
    0: 'วันจันทร์',
    1: 'วันอังคาร', 
    2: 'วันพุธ',
    3: 'วันพหัสบดี',
    4: 'วันศุกร์',
    5: 'วันเสาร์',
    6: 'วันอาทิตย์'
}

# เพิ่มชื่อวันในสัปดาห์
df_combined['day_name'] = df_combined['วันที่'].dt.weekday.map(day_names_thai)

# หาคอลัมน์ subject ใน calendar
subject_col_name = None
for col in calendar.columns:
    if 'subject' in col.lower() or 'ชื่อ' in col.lower() or 'เรื่อง' in col.lower():
        subject_col_name = col
        break

print(f"ใช้คอลัมน์ subject: {subject_col_name}")

# เพิ่มข้อมูลวันหยุดและเทศกาล
def get_holiday_info(date):
    holiday = calendar[calendar[date_col_name].dt.date == date.date()]
    if not holiday.empty:
        holiday_type = holiday[category_col_name].iloc[0] if category_col_name else 'normal'
        holiday_subject = holiday[subject_col_name].iloc[0] if subject_col_name else 'normal'
        return pd.Series({
            'holiday_type': holiday_type,
            'holiday_subject': holiday_subject
        })
    return pd.Series({
        'holiday_type': 'normal',
        'holiday_subject': 'normal'
    })

holiday_info = df_combined['วันที่'].apply(get_holiday_info)
df_combined = pd.concat([df_combined, holiday_info], axis=1)

# จัดเรียงคอลัมน์
date_cols = ['วันที่', 'day_name', 'day_type', 'holiday_type', 'holiday_subject']
other_cols = [col for col in df_combined.columns if col not in date_cols]
df_combined = df_combined[date_cols + other_cols]

print("ตัวอย่าง df_combined:")
display(df_combined.head(10))

# แสดงตัวอย่างวันหยุดและเทศกาล
holidays_festivals = df_combined[df_combined['day_type'].isin([0, 2])]
if len(holidays_festivals) > 0:
    print(f"\nตัวอย่างวันหยุดและเทศกาล (5 วันแรก):")
    display(holidays_festivals[['วันที่', 'day_name', 'day_type', 'holiday_type', 'holiday_subject']].head())

# สถิติแยกตามประเภทวัน
print("\nสถิติการเดินทางแยกตามประเภทวัน:")
numeric_cols = df_combined.select_dtypes(include=['number']).columns
stats = df_combined.groupby('day_type')[numeric_cols].agg(['mean', 'std']).round(2)
if len(stats) > 0:
    day_type_mapping = {0: 'Holiday', 1: 'Normal', 2: 'Festival'}
    stats.index = [day_type_mapping.get(idx, f'Type_{idx}') for idx in stats.index]
    display(stats)

ใช้คอลัมน์ subject: Subject
ตัวอย่าง df_combined:


,วันที่,day_name,day_type,holiday_type,holiday_subject,ยานพาหนะ/ท่า,ปริมาณ
0,2020-01-01,วันพุธ,0,holiday,วันขึ้นปีใหม่,รถไฟฟ้าสายสีน้ำเงิน,236053.0
1,2020-01-01,วันพุธ,0,holiday,วันขึ้นปีใหม่,รถไฟฟ้าสายสีม่วง,23135.0
2,2020-01-01,วันพุธ,0,holiday,วันขึ้นปีใหม่,รถไฟฟ้า ARL,48790.0
3,2020-01-01,วันพุธ,0,holiday,วันขึ้นปีใหม่,รถไฟฟ้า BTS,461400.0
4,2020-01-02,วันพหัสบดี,1,normal,normal,รถไฟฟ้าสายสีน้ำเงิน,341059.0
5,2020-01-02,วันพหัสบดี,1,normal,normal,รถไฟฟ้าสายสีม่วง,39445.0
6,2020-01-02,วันพหัสบดี,1,normal,normal,รถไฟฟ้า ARL,65090.0
7,2020-01-02,วันพหัสบดี,1,normal,normal,รถไฟฟ้า BTS,720500.0
8,2020-01-03,วันศุกร์,1,normal,normal,รถไฟฟ้าสายสีน้ำเงิน,373005.0
9,2020-01-03,วันศุกร์,1,normal,normal,รถไฟฟ้าสายสีม่วง,43185.0



ตัวอย่างวันหยุดและเทศกาล (5 วันแรก):


,วันที่,day_name,day_type,holiday_type,holiday_subject
0,2020-01-01,วันพุธ,0,holiday,วันขึ้นปีใหม่
1,2020-01-01,วันพุธ,0,holiday,วันขึ้นปีใหม่
2,2020-01-01,วันพุธ,0,holiday,วันขึ้นปีใหม่
3,2020-01-01,วันพุธ,0,holiday,วันขึ้นปีใหม่
12,2020-01-04,วันเสาร์,0,normal,normal



สถิติการเดินทางแยกตามประเภทวัน:


day_type          ปริมาณ           
             mean  std       mean        std
Holiday       0.0  0.0  138800.78  178903.55
Normal        1.0  0.0  214764.57  262563.10
Festival      2.0  0.0  208003.28  262960.32

In [7]:
df_combined[df_combined['day_type'] == 2]

,วันที่,day_name,day_type,holiday_type,holiday_subject,ยานพาหนะ/ท่า,ปริมาณ
40,2020-01-11,วันเสาร์,2,festival,วันเด็ก,รถไฟฟ้าสายสีน้ำเงิน,304673.0
41,2020-01-11,วันเสาร์,2,festival,วันเด็ก,รถไฟฟ้าสายสีม่วง,36728.0
42,2020-01-11,วันเสาร์,2,festival,วันเด็ก,รถไฟฟ้า ARL,64336.0
43,2020-01-11,วันเสาร์,2,festival,วันเด็ก,รถไฟฟ้า BTS,698600.0
176,2020-02-14,วันศุกร์,2,festival,วาเลนไทน์,รถไฟฟ้าสายสีน้ำเงิน,432048.0
...,...,...,...,...,...,...,...
9759,2025-02-14,วันศุกร์,2,festival,วาเลนไทน์,รถไฟฟ้าสายสีเหลือง,57127.0
9760,2025-02-14,วันศุกร์,2,festival,วาเลนไทน์,รถไฟฟ้าสายสีชมพู,78652.0
9761,2025-02-14,วันศุกร์,2,festival,วาเลนไทน์,รถไฟฟ้า ARL,89824.0
9762,2025-02-14,วันศุกร์,2,festival,วาเลนไทน์,รถไฟฟ้า BTS,972455.0


In [8]:
df_combined['ยานพาหนะ/ท่า'].unique() 

array(['รถไฟฟ้าสายสีน้ำเงิน', 'รถไฟฟ้าสายสีม่วง', 'รถไฟฟ้า ARL',
       'รถไฟฟ้า BTS', 'รถไฟฟ้าสายสีแดง', 'รถไฟฟ้าสายสีเหลือง',
       'รถไฟฟ้าสายสีชมพู'], dtype=object)

In [9]:
df_combined.head(10)

,วันที่,day_name,day_type,holiday_type,holiday_subject,ยานพาหนะ/ท่า,ปริมาณ
0,2020-01-01,วันพุธ,0,holiday,วันขึ้นปีใหม่,รถไฟฟ้าสายสีน้ำเงิน,236053.0
1,2020-01-01,วันพุธ,0,holiday,วันขึ้นปีใหม่,รถไฟฟ้าสายสีม่วง,23135.0
2,2020-01-01,วันพุธ,0,holiday,วันขึ้นปีใหม่,รถไฟฟ้า ARL,48790.0
3,2020-01-01,วันพุธ,0,holiday,วันขึ้นปีใหม่,รถไฟฟ้า BTS,461400.0
4,2020-01-02,วันพหัสบดี,1,normal,normal,รถไฟฟ้าสายสีน้ำเงิน,341059.0
5,2020-01-02,วันพหัสบดี,1,normal,normal,รถไฟฟ้าสายสีม่วง,39445.0
6,2020-01-02,วันพหัสบดี,1,normal,normal,รถไฟฟ้า ARL,65090.0
7,2020-01-02,วันพหัสบดี,1,normal,normal,รถไฟฟ้า BTS,720500.0
8,2020-01-03,วันศุกร์,1,normal,normal,รถไฟฟ้าสายสีน้ำเงิน,373005.0
9,2020-01-03,วันศุกร์,1,normal,normal,รถไฟฟ้าสายสีม่วง,43185.0


In [10]:
df_combined.to_csv('../data/daily_transport_rail_with_special_day.csv', index = False)